## AES-128 Trace Capture

In [ ]:
# capture using FOBOS3
# requires PYNQ FOBOS3 board
# requires FOBOS3 shield
import os
import sys
import numpy as np
import shutil
import json
import foboslib as fb
from foboslib.capture.ctrl.pynqctrl import PYNQCtrl
from foboslib.common.projmgr import ProjectManager
from foboslib.capture.fobosTVGen import FobosTVGen

In [ ]:
# Acquisition Configuration
cfg                     = {}

# Connection to bontrol board
cfg['ip']               = '192.168.10.99'
cfg['port']             = 9995

# Project
cfg['workspace']        = "fobosworkspace"       # Folder to store projects
cfg['projectName']      = "aes"                  # Name of this project. Files will be in workspace/projectName/
cfg['bitFile']          = 'half_duplex_dut.bit'

# Data files
cfg['keyFile']          = "key.txt"              # key used for encryption / decryption
cfg['bitFile']          = "half_duplex_dut.bit"  # Bitstream for programming the DUT (target) 
cfg['plainFile']        = "plaintext.txt"        # plaintext in hex
cfg['dinFile']          = "dinFile.txt"          # test vectors in FOBOS format incl. plaintext and key
cfg['cipherFile']       = "ciphertext.txt"       # observed ciphertext, result from applying test vectors
cfg['traceFile']        = "powerTraces.npy"      # Name of file for recording power traces

# Cipher-specific parameters
cfg['blockSize']        = 16                     # size of a block of plaintext in bytes
cfg['cipherSize']       = 16                     # size of a block of ciphertext in bytes
cfg['keySize']          = 16                     # size of the key in bytes

In [ ]:
#Acquistion/scope configuration
cfg['traceNum']         = 100                   # number of traces to run
cfg['DUTClk']           = 1                     # clock frequency of the DUT in KHz, [range: 1000 - 100000]
cfg['samplingFreq']     = 50                    # sampling frequency of the Oscilloscope in Msps [default: 50][range: 1 - 100]
cfg['samplesPerTrace']  = 1000                  # number of sample in one trace [range: 1 - 2^17]
cfg['ADCGain']          = 60                     # amplification of ADC input signal [default: 40][range: 0 - 60]
cfg['ADCHiLo']          = 1

In [ ]:
# Configure project directories
pm = ProjectManager()
pm.setWorkSpaceDir(cfg['workspace'])
pm.setProjName(cfg['projectName'])
projDir = pm.getProjDir()

In [ ]:
# Generate test vectors
tvGen = FobosTVGen(traceNum=cfg['traceNum'],
                   blockSize=cfg['blockSize'],
                   keySize=cfg['keySize'],
                   cipherSize=cfg['cipherSize'],
                   dinFile= os.path.join(projDir, cfg['dinFile']),
                   plaintextFile=os.path.join(projDir, cfg['plainFile']),
                   keyFile=os.path.join(projDir, cfg['keyFile'])
                   )
tvGen.generateTVs()

In [ ]:
ctrl = PYNQCtrl(cfg['ip'], cfg['port'])

In [ ]:
ctrl.setDUTClk(cfg['DUTClk'])
ctrl.setDUTInterface(fb.INTERFACE_4BIT)
ctrl.setOutLen(cfg['cipherSize'])
ctrl.setTriggerMode(fb.TRG_FULL)
ctrl.setSamplingFrequency(cfg['samplingFreq'])
ctrl.setADCGain(cfg['ADCGain'])
ctrl.setADCHiLo(cfg['ADCHiLo'])
ctrl.setSamplesPerTrace(cfg['samplesPerTrace'])

In [ ]:
# Create capture directory
captureDir          = pm.getCaptureDir()
# Copy data and configuration files to capture directory
tvFileName          = os.path.join(projDir, cfg['dinFile'])
plainFileName       = os.path.join(projDir, cfg['plainFile'])
cipherFileName      = os.path.join(captureDir, cfg['cipherFile'])
traceFileName       = os.path.join(captureDir, cfg['traceFile'])
tvFile              = open(tvFileName, "r")
cipherFile          = open(cipherFileName, "w")
traceFile           = open(traceFileName, "a+b")
shutil.copy(tvFileName, captureDir)
shutil.copy(plainFileName, captureDir)
# save config to a file
configFile = open(os.path.join(captureDir, 'acquisitionConfig.json'), "w")
configFile.write(json.dumps(cfg, indent=4))

In [ ]:
# Trace collection loop
print('Processing test vectors ...')
traceNum = 0
while traceNum < cfg['traceNum']:
    data = tvFile.readline()
    status, result, trace = ctrl.processData2(data, cfg['cipherSize'])
    cipherFile.write(result + "\n")
    np.save(traceFile, trace)
    if traceNum % 100 == 0:
        sys.stdout.write('Progress:' + "{:.2f}".format(traceNum/cfg['traceNum']*100) + '%\r')
        sys.stdout.flush()
    traceNum += 1
print('Data acquisition complete.')


In [ ]:
# Close IO
ctrl.disconnect()
traceFile.close()
cipherFile.close()
tvFile.close()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
fig.patch.set_facecolor('white')
plt.rcParams.update({'font.size': 18})
traceFile = open(traceFileName, "r+b")
maxtrace = 1
plt.figure(figsize=(10,8))
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.title('Captured Traces')
for i in range(min(maxtrace, cfg['traceNum'])):
    trace = np.load(traceFile)
    plt.plot(trace)

plt.savefig(os.path.join(captureDir, 'traces.png'),facecolor=fig.get_facecolor())
# plt.close()
traceFile.close()